<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/LLM_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_%EA%B0%9C%EB%85%90%EA%B3%BC_%EA%B8%B0%EB%B3%B8_%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#  Lima-3 --> 경량화 모델
model_name = "facebook/opt-125m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#pip install peft
# LoRa 설정 정의
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r = 8 # lora가 추가하는 저랭크 행렬의 차원  # W = W + W'
    ,lora_alpha=16
    ,target_modules=['q_proj','v_proj'] # query, value projection에만 lora적용
    ,lora_dropout=0.1
    ,bias="none"
    ,task_type='CAUSAL_LM' # gpt류 모델에서 텍스트 생성과제 지정
)
# LoRa 모델 적용
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 125,534,208 || trainable%: 0.2349


QLora
  Lora구조에 4비트 양자화를 추가 메모리 사용량 줄임

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch
# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True
    ,bnb_4bit_quant_type='nf4'  # 실수형 4비트로 정규화
    ,bnb_4bit_compute_dtype=torch.bfloat16 # 연사용 데이터 타입
)
# 모델 로드(양자화 적용)
model = AutoModelForCausalLM.from_pretrained(
    model_name
    ,quantization_config=bnb_config
    ,device_map='auto'
)
# 메모리 사용량 확인

print(f"memory : {model.get_memory_footprint()/1e9:.2f}GB")

memory : 0.12GB


RoRA 사용 전후 비교

In [ ]:
from datasets import Dataset
#간단한QA 데이터  -> 실제는 대규모 데이터
data = [
    {'question':'파인튜인이란 무엇인가?', 'answer':'파인튜닝은 사전 학습된 모델을 특정 작업에 맞게 추가 학습시키는 과정입니다.'}
    ,{'question':'LoRA의 장점은?', 'answer':'LoRA는 소수의 파라미터만 학습하여 메모리와 시간을 절약합니다.'}
    ,{'question':'QLoRA의 장점은?','answer':' QLoRA는 메모리 유지 관리 요구 사항이 적은 것이 주된 장점인 기술입니다. LoRA와 마찬가지로 효율성을 중시하여 미세 조정 학습 프로세스를 더 쉽고 빠르게 진행할 수 있도록 합니다'}
]
dataset = Dataset.from_list(data)
print(dataset)
#데이터 포멧팅
def format_dataset(data):
  return {'text':f"<s>[INST] {data['question']}[/INST] {data['answer']}</s>"}
dataset = dataset.map(format_dataset)

Dataset({
    features: ['question', 'answer'],
    num_rows: 3
})


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# 모델과 토크나이져 로드
from transformers import AutoTokenizer, AutoModelForCausalLM
#  Lima-3 --> 경량화 모델
model_name = "facebook/opt-125m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# LoRA 적용전
prompt = "파인튜닝이란 무엇인가?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs,
                         max_length = 50,
                         do_sample = True,
                         top_p = 0.95,
                         temperature=0.7,
                         pad_token_id = tokenizer.pad_token_id)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

파인튜닝이란 무엇인가?

적지을 보력을 보�


LoRA 튜닝 적용

In [ ]:
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import Dataset
#간단한QA 데이터  -> 실제는 대규모 데이터
data = [
    {'question':'파인튜인이란 무엇인가?', 'answer':'파인튜닝은 사전 학습된 모델을 특정 작업에 맞게 추가 학습시키는 과정입니다.'}
    ,{'question':'LoRA의 장점은?', 'answer':'LoRA는 소수의 파라미터만 학습하여 메모리와 시간을 절약합니다.'}
    ,{'question':'QLoRA의 장점은?','answer':' QLoRA는 메모리 유지 관리 요구 사항이 적은 것이 주된 장점인 기술입니다. LoRA와 마찬가지로 효율성을 중시하여 미세 조정 학습 프로세스를 더 쉽고 빠르게 진행할 수 있도록 합니다'}
]
data += [
    {'question': 'Transfer Learning이란?', 'answer': 'Transfer Learning은 기존에 학습된 모델의 지식을 새로운 작업에 적용하는 기법입니다.'},
    {'question': '사전 학습 모델(Pretrained Model)의 장점은?', 'answer': '사전 학습 모델은 방대한 데이터로 미리 학습되어 적은 데이터로도 높은 성능을 낼 수 있습니다.'},
    {'question': 'GPU가 딥러닝에서 중요한 이유는?', 'answer': 'GPU는 병렬 연산에 강해 딥러닝 모델의 대량 연산을 빠르게 처리할 수 있습니다.'},
    {'question': '학습률(Learning Rate)이란?', 'answer': '학습률은 모델이 얼마나 빠르게 최적값을 향해 나아가는지를 결정하는 하이퍼파라미터입니다.'},
    {'question': 'Overfitting이란?', 'answer': 'Overfitting은 모델이 학습 데이터에 과하게 맞춰져 새로운 데이터에 일반화되지 못하는 현상입니다.'},
    {'question': 'Dropout의 역할은?', 'answer': 'Dropout은 학습 시 일부 뉴런을 임의로 제거하여 과적합을 방지하는 기법입니다.'},
    {'question': 'Batch Normalization의 효과는?', 'answer': 'Batch Normalization은 내부 공변량 변화 문제를 줄여 학습을 안정화하고 빠르게 만듭니다.'},
    {'question': 'Tokenizer의 역할은?', 'answer': 'Tokenizer는 텍스트를 모델이 처리할 수 있는 형태의 토큰으로 변환하는 도구입니다.'},
    {'question': 'Attention Mechanism이란?', 'answer': 'Attention Mechanism은 입력 시퀀스의 중요한 부분에 집중하여 더 나은 출력을 생성하는 기법입니다.'},
    {'question': 'Prompt Engineering이란?', 'answer': 'Prompt Engineering은 원하는 출력을 얻기 위해 모델에 주는 입력 문장을 정교하게 설계하는 기술입니다.'}
]


In [ ]:
# 1. 환경 설정
import os
import torch
import transformers
import peft
import datasets
import trl
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from datasets import Dataset

# # 라이브러리 설치
# !pip install torch==2.3.0 transformers==4.45.0 peft==0.12.0 datasets==2.20.0 trl==0.9.6

# # 버전 확인
# print(f"PyTorch: {torch.__version__}")
# print(f"Transformers: {transformers.__version__}")
# print(f"PEFT: {peft.__version__}")
# print(f"Datasets: {datasets.__version__}")
# print(f"TRL: {trl.__version__}")

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. 데이터 준비
data = [
    {"question": "파인튜닝이란 무엇인가?", "answer": "파인튜닝은 사전 학습된 모델을 특정 작업에 맞게 추가 학습시키는 과정입니다."},
    {"question": "LoRA의 장점은?", "answer": "LoRA는 소수의 파라미터만 학습하여 메모리와 시간을 절약합니다."},
    {"question": "한국어 LLM의 도전 과제는?", "answer": "한국어 데이터 부족과 토크나이저 최적화 문제입니다."}
]
dataset = Dataset.from_list(data)

# ChatML 포맷팅 함수
def formatting_func(example):
    return f"<|im_start|>user\n{example['question']}<|im_end|>\n<|im_start|>assistant\n{example['answer']}<|im_end|>"

# 3. 모델 및 토크나이저 로드
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.config.pad_token_id = tokenizer.pad_token_id

# 설정 확인
print(f"Tokenizer pad_token_id: {tokenizer.pad_token_id} (type: {type(tokenizer.pad_token_id)})")
print(f"Model config pad_token_id: {model.config.pad_token_id}")

# 4. LoRA 파인튜닝
# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=10,
    logging_steps=1,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# SFTTrainer 설정
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    formatting_func=formatting_func,
)

# 학습 실행
trainer.train()

# 모델 및 토크나이저 저장
lora_model_path = "./lora_finetuned_model"
model.save_pretrained(lora_model_path)
tokenizer.save_pretrained(lora_model_path)

# 5. LoRA 적용 후 텍스트 생성
# LoRA 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(lora_model_path, local_files_only=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = PeftModel.from_pretrained(
    base_model,
    lora_model_path,
    local_files_only=True
)
model.config.pad_token_id = tokenizer.pad_token_id

# 테스트 프롬프트 1
prompt = "파인튜닝이란 무엇인가?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=50,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id
)
print("=== LoRA 적용 후 출력 ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# 테스트 프롬프트 2
prompt_2 = "LoRA의 장점은?"
inputs = tokenizer(prompt_2, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_length=50,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    pad_token_id=tokenizer.pad_token_id
)
print("=== LoRA 적용 후 출력 (질문 2) ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Using device: cuda


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenizer pad_token_id: 1 (type: <class 'int'>)
Model config pad_token_id: 1


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: The `formatting_func` should return a list of processed strings since it can lead to silent bugs.